# Capstone Journal

**5/15/2025**:   Class meetup. Read, set, go!

**5/19/2025**:   Reviewed requirements, created project directory, base files, and drafted a vision board.

![Management Cycle Flowchart](management-cycle.png)

- NOTE: I want to give the CSV data some context by generating a new fake csv that reflects a developers finances.
- ACTION: Created `csv_faker.py` to generate 500,000 transaction records and saved them to `developer_transactions.csv`.

In [6]:
# Task 1: Loading Transactions from a CSV File

# Parse data with datetime.strptime
# Make amount negative for 'debit'
# Convert transaction_id and customer_id to integers
# Create dictionary with all fields
# Add to transactions 
# Catch FileNotFoundError and ValueError

# NOTE: These functions will all go into a `FinanceUtils` class inside the utils.py file

import csv
from datetime import datetime

def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""

    transactions = []
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Convert transaction ID to integer
            row['transaction_id'] = int(row['transaction_id'])
            # Convert customer ID to integer
            row['customer_id'] = int(row['customer_id'])
            # Convert date string to datetime object
            row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
            # Convert amount to float
            row['amount'] = float(row['amount'])
            # Make amount negative for 'debit'
            if row['type'] == 'debit':
                row['amount'] = -row['amount']
            transactions.append(row)

    return transactions

# Test: Print the first 3 transactions
print("Loaded transactions:")
transactions = load_transactions(filename='developer_transactions.csv')
for transaction in range(3):
    print(transactions[transaction])

print(f"Total transactions loaded: {len(transactions)}")

Loaded transactions:
{'transaction_id': 1, 'date': datetime.date(2021, 11, 19), 'customer_id': 719, 'amount': 363.2, 'type': 'transfer', 'description': 'Payment to friend for dinner'}
{'transaction_id': 2, 'date': datetime.date(2024, 4, 16), 'customer_id': 674, 'amount': 456.13, 'type': 'transfer', 'description': 'Reimbursement for shared purchase'}
{'transaction_id': 3, 'date': datetime.date(2017, 4, 13), 'customer_id': 651, 'amount': 273.71, 'type': 'transfer', 'description': 'Transfer to investment account'}
Total transactions loaded: 500000


In [ ]:
# Class method:

import csv
from datetime import datetime
import logging
import os

class FinanceUtils:
    """Class to manage financial transactions with CRUD operations and analysis."""

    # Class constructor

    def load_transactions(self, filename='financial_transactions.csv'):
        """
        Load transactions from a CSV file into self.transactions.
        
        Args:
            filename (str): Path to the CSV file.
            
        Returns:
            bool: True if loading succeeds, False otherwise.
        """
        self.transactions = []
        required_columns = {'transaction_id', 'date', 'customer_id', 'amount', 'type', 'description'}

        try:
            with open(filename, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)

                # Check required columns
                if not required_columns.issubset(reader.fieldnames):
                    missing = required_columns - set(reader.fieldnames)
                    logging.error(f"Missing columns in CSV: {missing}")
                    print(f"Missing columns in CSV: {missing}")
                    return False
                
                for row_num, row in enumerate(reader, start=2):
                    try:
                        # Validate transaction_id
                        try:
                            transaction_id = int(row['transaction_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid transaction_id '{row['transaction_id']}'")
                            continue

                        # Validate date
                        date_str = row['date'].strip()
                        try:
                            date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid date format '{date_str}'")
                            continue

                        # Validate customer_id
                        try:
                            customer_id = int(row['customer_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid customer_id '{row['customer_id']}'")
                            continue

                        # Validate amount
                        try:
                            amount = float(row['amount'])
                            if amount < 0:
                                logging.error(f"Row {row_num}: Negative amount '{amount}'")
                                continue

                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid amount '{row['amount']}'")
                            continue

                        # Validate type
                        transaction_type = row['type'].strip().lower()
                        if transaction_type not in {'credit', 'debit', 'transfer'}:
                            logging.error(f"Row {row_num}: Invalid transaction type '{transaction_type}'")
                            continue

                        # Adjust amount for debit
                        if transaction_type == 'debit':
                            amount = -amount

                        # Validate description
                        description = row.get('description')
                        if description is None or not str(description).strip():
                            logging.error(f"Row {row_num}: Empty description")
                            continue
                        description = str(description).strip()

                        # Create transaction dictionary
                        transaction = {
                            'transaction_id': transaction_id,
                            'date': date_obj,
                            'customer_id': customer_id,
                            'amount': amount,
                            'type': transaction_type,
                            'description': description
                        }
                        self.transactions.append(transaction)

                    except KeyError as e:
                        logging.error(f"Row {row_num}: Missing column {e}")
                        continue

                print(f"Loaded {len(self.transactions)} transactions from '{filename}'.")

                # Create a backup of the original file and save it with a timestamp to /snapshots
                if not os.path.exists('snapshots'):
                    os.makedirs('snapshots')

                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                backup_filename = os.path.join('snapshots', f'backup_{timestamp}.csv')
                try:
                    with open(filename, 'rb') as src_file, open(backup_filename, 'wb') as dst_file:
                        dst_file.write(src_file.read())
                    print(f"Backup created: '{backup_filename}'")
                except Exception as e:
                    logging.error(f"Failed to create backup: {e}")

                return True
            
        except FileNotFoundError:
            logging.error(f"File '{filename}' not found.")
            print(f"File '{filename}' not found.")
            return False
        
        except csv.Error:
            logging.error(f"Malformed CSV file '{filename}'.")
            print(f"Error reading CSV file '{filename}'.")
            return False
        
        except IOError as e:
            logging.error(f"IO error reading {filename}: {e}")
            print(f"Error: IO error reading file: {e}")
            return False
        


## Testing

✅ Skips rows with invalid fields and prints a warning.  
✅ Loads and counts transactions accurately.  
✅ Checks required columns  
✅ Validates transaction_id  
✅ Validates date  
✅ Validates customer_id  
✅ Validates amount  
✅ Validates type  
✅ Validates description  

✅ Backup CSV snapshots  
✅ Handle backup errors  



**5/21/2025**: Starting Task 2 and implementing CRUD features.  

**Goals**:  
- Reject invalid transaction types
- Add an option to filter by types when viewing table
- Suggest customer IDs from existing transactions
- Format dates as "Oct 26, 2020" 

Sticking with class methods for my finance utilities and pulling in a library for formatting pretty tables.

I'll write these methods and then test the terminal interface for usability and readability. Colors? Hm.

In [ ]:
# Example functions

def add_transaction(transactions):
    """Add a new transaction from user input."""
    # Prompt for date, customer_id, amount, type, description
    # Validate date, amount, type
    # Generate new transaction_id
    # Create dictionary and append
    pass


def view_transactions(transactions):
    """Display transactions in a table."""
    # Print header
    # Loop through transactions
    # Format each row
    pass

In [ ]:
# Class methods

# Using the `tabulate` library for better table formatting
from tabulate import tabulate

def add_transaction(self):
    print("\nAdd New Transaction (enter 'cancel' to abort)")

    # Date input
    while True:
        date_input = input("Enter date (YYYY-MM-DD): ").strip()
        if date_input.lower() == 'cancel':
            print("Transaction addition cancelled.")
            return False
        try:
            date_obj = datetime.strptime(date_input, '%Y-%m-%d').date()
            break
        except ValueError:
            logging.error(f"Invalid date format: {date_input}")
            print("Invalid date format. Please enter in YYYY-MM-DD format.")

    # Customer ID input with suggestions
    customer_ids = sorted(set(t['customer_id'] for t in self.transactions))
    if customer_ids:
        print(f"Valid customer IDs: {', '.join(map(str, customer_ids[:10]))}{'...' if len(customer_ids) > 10 else ''}")
    while True:
        customer_input = input("Enter customer ID (integer): ").strip()
        if customer_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        try:
            customer_id = int(customer_input)
            break
        except ValueError:
            logging.error(f"Invalid customer ID input: {customer_input}")
            print("Error: Customer ID must be an integer. Please try again.")

    # Amount input
    while True:
        amount_input = input("Enter amount (positive number): ").strip()
        if amount_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        try:
            amount = float(amount_input)
            if amount <= 0:
                logging.error(f"Non-positive amount input: {amount_input}")
                print("Error: Amount must be positive. Please try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid amount input: {amount_input}")
            print("Error: Amount must be a number. Please try again.")

    # Type input
    valid_types = {'credit', 'debit', 'transfer'}
    while True:
        type_input = input("Enter type (credit/debit/transfer): ").strip().lower()
        if type_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        if type_input not in valid_types:
            logging.error(f"Invalid transaction type input: {type_input}")
            print(f"Error: Type must be one of {', '.join(valid_types)}. Please try again.")
            continue
        break

    # Adjust amount for debit
    if type_input == 'debit':
        amount = -amount

    # Description input
    while True:
        description = input("Enter description: (non-empty): ").strip()
        if description.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        if not description:
            logging.error("Empty description input")
            print("Error: Description cannot be empty. Please try again.")
            continue
        break

    # Generate new transaction ID
    transaction_id = max((t[transaction_id] for t in self.transactions), default=0) + 1

    # Create and append transaction
    transaction = {
        'transaction_id': transaction_id,
        'date': date_obj,
        'customer_id': customer_id,
        'amount': amount,
        'type': type_input,
        'description': description
    }
    self.transactions.append(transaction)
    print(f"Transaction {transaction} added successfully!")
    return True

def view_transactions(self, filter_type=None):
    if not self.transactions:
        print("No transactions to display.")
        return False
    
    # Apply filter
    valid_types = {'credit', 'debit', 'transfer'}
    if filter_type and filter_type.lower() not in valid_types:
        logging.error(f"Invalid filter type: {filter_type}")
        print(f"Error: Filter type must be one of {', '.join(valid_types)} or empty.")
        return False
    
    transactions = (
        [t for t in self.transactions if t['type'] == filter_type.lower()]
        if filter_type else self.transactions
    )

    if not transactions:
        print(f"No {filter_type} transactions found." if filter_type else "No transactions found.")
        return False
    
    # Prepare table data
    table = [
        [
            t['transaction_id'],
            t['date'].strftime('%b %d, %Y'),
            t['customer_id'],
            f"${t['amount']:,.2f}",
            t['type'].capitalize(),
            t['description'][:30] + ('...' if len(t['description']) > 30 else '')
        ]
        for t in transactions
    ]

    headers = ['ID', 'Date', 'Customer', 'Amount', 'Type', 'Description']
    print(f"\n{'Filtered' if filter_type else 'All'} Transactions:")
    print(tabulate(table, headers=headers, tablefmt='grid', stralign='left'))
    return True

**Notes**:  
Should I add pagination to the table results?  
Users can enter a negative customer ID when adding transactions; should always be positive.

**Tests**:  
✅ Test Invalid Inputs (Date, Customer ID, Amount, Type, Description)  
✅ Test Type Filters 

**Optional Tweaks**:  
- Paginate with navigation prompts
- Added year filter
- Validate `transaction_id` uniqueness
- Enforce positive `customer_id`: Reject negative values and log errors

**More Tests**:  
✅ Test Type/Year Filters  
✅ Test Non-Positive customer_id  
✅ Test Pagination Navigation  
✅ Test Transaction Views w/ Filters  
✅ Test Empty CSV

**5/22/2025**: Starting Task 3 and continuing CRUD features.  

**Example Code**  

```python
def update_transaction(transactions):
    """Update a transaction’s details."""
    # Show transactions with numbers
    # Ask user to pick a number
    # Ask which field to change
    # Update field
    pass

def delete_transaction(transactions):
    """Delete a transaction."""
    # Show transactions with numbers
    # Ask user to pick a number
    # Confirm and remove
    pass
```  

**Goals**:  
- Use enumerate for transaction numbers
- Validate transaction number input
- Confirm deletions with "Are you sure?"
- Prevent invalid type updates in update_transaction
- Show transaction details before deletion
- Allow updating multiple fields at once
- Add a cancel option for updates/deletions

In [ ]:
# Class method
def _get_transaction_by_id(self, transaction_id):
    """Helper method to find a transaction by its ID."""
    for t in self.transactions:
        if t['transaction_id'] == transaction_id:
            return t
    return None

def update_transaction(self):
    if not self.transactions:
        print("No transactions to update.")
        return False
    
    print("\nUpdate Transaction (enter 'cancel' to abort)")
    while True:
        id_input = input("Enter transaction ID (e.g., 123): ").strip()
        if id_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        try:
            transaction_id = int(id_input)
            transaction = self._get_transaction_by_id(transaction_id)
            if not transaction:
                logging.error(f"Transaction ID {transaction_id} not found.")
                print(f"Error: Transaction ID {transaction_id} not found. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid transaction ID input: {id_input}")
            print("Error: Transaction ID must be an integer. Please try again.")

    # Display current transaction
    print(f"\nUpdating Transaction {transaction_id}:")
    print(f"  Current: {transaction['date'].strftime('%Y-%m-%d')}, Customer {transaction['customer_id']}, "
            f"${abs(transaction['amount']):,.2f} {transaction['type'].capitalize()}, {transaction['description']}")
    print("Enter new values (press Enter to keep current, 'cancel' to abort)")

    # Date input
    while True:
        date_input = input(f"New date [{transaction['date'].strftime('%Y-%m-%d')}]: ").strip()
        if date_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not date_input:
            date_obj = transaction['date']
            break
        try:
            date_obj = datetime.strptime(date_input, '%Y-%m-%d').date()
            break
        except ValueError:
            logging.error(f"Invalid date input: '{date_input}'")
            print("Error: Date must be in YYYY-MM-DD format (e.g., 2020-10-26). Try again.")

    # Customer ID input
    customer_ids = sorted(set(t['customer_id'] for t in self.transactions if t['customer_id'] > 0))
    if customer_ids:
        print(f"Valid customer IDs: {', '.join(map(str, customer_ids[:10]))}{'...' if len(customer_ids) > 10 else ''}")
    while True:
        customer_input = input(f"New customer ID [{transaction['customer_id']}]: ").strip()
        if customer_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not customer_input:
            customer_id = transaction['customer_id']
            break
        try:
            customer_id = int(customer_input)
            if customer_id <= 0:
                logging.error(f"Non-positive customer ID input: '{customer_id}'")
                print("Error: Customer ID must be a positive integer. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid customer ID input: {customer_input}")
            print("Error: Customer ID must be a positive integer. Please try again.")

    # Amount input
    while True:
        amount_input = input(f"New amount [{abs(transaction['amount']):,.2f}]: ").strip()
        if amount_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not amount_input:
            amount = abs(transaction['amount'])
            break
        try:
            amount = float(amount_input)
            if amount <= 0:
                logging.error(f"Non-positive amount input: '{amount}'")
                print("Error: Amount must be positive. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid amount input: '{amount_input}'")
            print("Error: Amount must be a number. Try again.")

    # Type input
    valid_types = {'credit', 'debit', 'transfer'}
    while True:
        type_input = input(f"New type [{transaction['type']}]: ").strip().lower()
        if type_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not type_input:
            transaction_type = transaction['type']
            break
        if type_input not in valid_types:
            logging.error(f"Invalid transaction type input: '{type_input}'")
            print(f"Error: Type must be one of {', '.join(valid_types)}. Try again.")
            continue
        break

    # Adjust amount for debit
    if type_input == 'debit':
        amount = -amount

    # Description input
    while True:
        description = input(f"New description [{transaction['description']}]: ").strip()
        if description.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not description:
            description = transaction['description']
            break
        if not description.strip():
            logging.error("Empty description input")
            print("Error: Description cannot be empty. Try again.")
            continue
        description = description.strip()
        break

    # Update transaction
    transaction.update({
        'date': date_obj,
        'customer_id': customer_id,
        'amount': amount,
        'type': type_input,
        'description': description
    })
    print(f"Transaction {transaction_id} updated successfully!")
    return True

def delete_transaction(self):
    """
    Prompt user to delete a transaction by ID.
        
    Returns:
        bool: True if transaction is deleted, False if cancelled or invalid.
    """
    if not self.transactions:
        print("No transactions to delete.")
        return False

    print("\nDelete Transaction (enter 'cancel' to abort)")
    while True:
        id_input = input("Enter transaction ID (e.g., 123): ").strip()
        if id_input.lower() == 'cancel':
            print("Deletion cancelled.")
            return False
        try:
            transaction_id = int(id_input)
            transaction = self._get_transaction_by_id(transaction_id)
            if not transaction:
                logging.error(f"Transaction ID not found: '{transaction_id}'")
                print(f"Error: Transaction ID {transaction_id} not found. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid transaction ID input: '{id_input}'")
            print("Error: Transaction ID must be an integer. Try again.")

    # Display transaction
    print(f"\nTransaction to delete (ID {transaction_id}):")
    print(f"  {transaction['date'].strftime('%Y-%m-%d')}, Customer {transaction['customer_id']}, "
            f"${abs(transaction['amount']):,.2f} {transaction['type'].capitalize()}, {transaction['description']}")

    # Confirm deletion
    while True:
        confirm = input("Are you sure? (y/n): ").strip().lower()
        if confirm == 'n' or confirm == 'cancel':
            print("Deletion cancelled.")
            return False
        if confirm == 'y':
            break
        print("Please enter 'y', 'n', or 'cancel'.")

    # Delete transaction
    self.transactions.remove(transaction)
    print(f"Transaction {transaction_id} deleted successfully!")
    return True


**Notes**:  
Can I automate the tests with a single test file by simulating user inputs?

Yes, yes I can! Added `test_finance_utils.py` for current and future tests. 

Onto Task 4.

**5/23/2025**: Starting Task 4 

During the meetup last night it was mentioned that we should *not* be using Copilot to write our code. I use a toolkit of AIs that replace my old ways of coding..  

> **TASK -> ROADBLOCK -> RESEARCH (GOOGLE, STACK OVERFLOW) -> REFACTOR -> SUCCESS**

With AI tools, it looks more like this..

> **TASK -> ROADBLOCK -> RESEARCH (GROK, CoPilot) -> REFACTOR -> SUCCESS**

The benefits outweigh the pitfalls IMHO. I write my own code, collaborating with AI to gain insights and follow best practices. For learners, these tools can dramatically enhance the experience when used responsibly and ethically.  

That said, whenever it comes to a task requiring deep understanding of a subject (i.e., python fundamentals), let me emphasize:  
**AI Tools must be combined with mindful coding practices and a goal of learning the subject.** 

Yes, these tools will code for you and there will be times when that is practical. No, this isn't the time. Moving on!  

---

## Analyzing Financial Data

"Weeks 5 and 7 introduced dictionaries and data processing. Calculate metrics like total credits, debits, and transfers, and group by type or customer ID. Use Week 2’s loops, Week 1’s arithmetic, and Week 5’s dictionaries."  

**Code Hint**:  
```python
def analyze_finances(transactions):
    """Calculate and display financial summaries."""
    # Sum credits, debits, transfers
    # Group by type or customer_id
    # Print results
    pass
```

**Requirements**:  
- Sum amounts by type using a dictionary.
- Format numbers with `f"${value:.2f}`.
- Handle transfers separately (niether income nor expense).
- Show the customer with the highest debt amount.
- Calculate percentage of total amount by type.
- Analyze transactions from 2022 only.
- Save analysis to `analysis.txt`.

**Expected Output**:
```
Financial Summary:
Total Credits: $6478.39
Total Debits: $7969.68
Total Transfers: $0.00
Net Balance: $-1491.29
By Type:
  Credit: $6478.39
  Debit: $7969.68
```

Take-Away: 
The analysis should produce a formatted summary, get percentage totals by type, filter by year, and save the analysis to a file.  
Then, *for fun* I can produce a customer summary analysis that shows customer stats such as highest debit, credit, and transfer. 